In [6]:

from exasol.ai.text.extraction import *
from exasol.ai.text.extraction.extraction import Extraction
from exasol.ai.text.extraction import AbstractExtraction
from exasol.nb_connector.connections import open_pyexasol_connection
from exasol.ai.text.extraction.abstract_extraction import Output
from exasol.nb_connector.secret_store import Secrets
from exasol.nb_connector.ai_lab_config import AILabConfig as CKey
from exasol.nb_connector.language_container_activation import get_activation_sql
from exasol.nb_connector.connections import open_pyexasol_connection


class ExtractionWrapper(AbstractExtraction):

    def run(self, conf: Secrets) -> None:
        activation_sql = get_activation_sql(conf)
        with open_pyexasol_connection(ai_lab_config, compression=True) as connection:
            connection.execute(query=activation_sql)
            db_schema = conf.db_schema
            language_alias = "PYTHON3_TXAIE" #conf.get(CKey.language_alias)todo change once var exists in sectret store
            Extraction(
                extractor=self.extractor,
                output=self.output,
                defaults=self.defaults).run(
                connection,
                db_schema,
                language_alias,
            )
 
